# 단층 퍼셉트론(SLP)

## 회귀 분석_Abalone


### 파이썬 모듈 불러들이기

In [18]:
import numpy as np
import csv

np.random.seed(1234)

### 하이퍼파라미터 정의

In [20]:
RND_MEAN = 0
RND_STD = 0.003
LEARNING_RATE = 0.001

### 실험용 메인 함수

In [22]:
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

### 데이터 적재함수 정의

In [24]:
def load_abalone_dataset():
    global data, input_cnt, output_cnt

    with open('data/abalone.csv') as f:
        csvreader = csv.reader(f)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt + output_cnt])

    for idx, row in enumerate(rows):
        if row[0] == 'I':
            data[idx, 0] = 1
        if row[0] == 'M':
            data[idx, 1] = 1
        if row[0] == 'F':
            data[idx, 0] = 1

        data[idx, 3:] = row[1:]

### 파라미터 초기화 함수 정의

In [25]:
def init_model():
    global weight, bias, input_cnt, output_cnt

    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])

    bias = np.zeros([output_cnt])

### 학습 및 평가 함수 정의

In [26]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()

    for epoch in range(epoch_count):
        losses, accs = [], []

        for i in range(step_count):
            train_x, train_y = get_train_data(mb_size, i)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)

        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print(f"Epoch: {epoch+1}: loss={np.mean(losses):5.3f}, accuracy={np.mean(accs):5.3f}/{acc:5.3f}")

    final_acc = run_test(test_x, test_y)
    print(f"\nFinal Test: final accuray = {final_acc:5.3f}")

### 학습 및 평가 데이터 획득 함수 정의

In [27]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx

    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8 // mb_size)
    test_begin_idx = step_count * mb_size

    return step_count

In [28]:
def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt

    test_data = data[shuffle_map[test_begin_idx:]]

    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

In [29]:
def get_train_data(mb_size, idx):
    global data, shuffle_map, test_begin_idx, output_cnt

    if idx == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*idx:mb_size*(idx+1)]]
    
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]


In [30]:
def run_train(x, y):
    # 순전파
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)

    # 역전파
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)

    return loss, accuracy

In [31]:
  def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    
    return accuracy

### 단층 퍼셈트론에 대한 순전파 및 역전파 함수 정의

In [32]:
def forward_neuralnet(x):
    global weight, bias

    output = np.matmul(x, weight) + bias

    return output, x

In [33]:
def backprop_neuralnet(G_output, x):
    global weight, bias

    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

### 후처리 과정에 대한 순전파 및 역전파 함수 정의

In [34]:
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)

    return loss, diff

In [35]:
def backprop_postproc(G_loss, diff):
    shape = diff.shape

    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff

    return G_output

### 정확도 계산 함수 정의

In [36]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y) / y))

    return 1 - mdiff

# 단층 퍼셉트론 메인 함수 실행

In [37]:
abalone_exec()

Epoch: 1: loss=7.657, accuracy=0.797/0.808
Epoch: 2: loss=6.491, accuracy=0.812/0.816
Epoch: 3: loss=6.110, accuracy=0.818/0.843
Epoch: 4: loss=5.745, accuracy=0.824/0.809
Epoch: 5: loss=5.566, accuracy=0.827/0.838
Epoch: 6: loss=5.545, accuracy=0.827/0.833
Epoch: 7: loss=5.436, accuracy=0.828/0.851
Epoch: 8: loss=5.389, accuracy=0.829/0.846
Epoch: 9: loss=5.302, accuracy=0.831/0.779
Epoch: 10: loss=5.336, accuracy=0.831/0.841

Final Test: final accuray = 0.841


### 파라미터 확인

In [38]:
print('weight:\n', weight)
print('\nbias: ', bias)

weight:
 [[ 8.04666320e-01]
 [ 1.51136359e+00]
 [ 2.86377226e-03]
 [ 4.50174061e+00]
 [ 5.92142818e+00]
 [ 5.48676574e+00]
 [ 5.33022695e+00]
 [-1.66660744e+01]
 [-2.40810520e+00]
 [ 1.23454553e+01]]

bias:  [2.3115759]


### 새로운 입력 벡터 X에 대한 예측

In [39]:
X = np.array([0,1,0,0.44,0.3,0.08,0.5,0.23,0.11,0.2])
output = forward_neuralnet(X)
print(output)

(array([9.05519092]), array([0.  , 1.  , 0.  , 0.44, 0.3 , 0.08, 0.5 , 0.23, 0.11, 0.2 ]))


### 하이퍼파라미터를 수정하며 실험

* LEARNING_RATE = 0.001 -> 0.1
* epoch_count = 10 -> 100
* mb_size = 10 -> 100

In [40]:
LEARNING_RATE = 0.1

epoch_count = 100
mb_size = 100
report = 20

abalone_exec(epoch_count=epoch_count, mb_size=mb_size, report=report)

Epoch: 20: loss=5.884, accuracy=0.821/0.821
Epoch: 40: loss=5.360, accuracy=0.831/0.833
Epoch: 60: loss=5.150, accuracy=0.834/0.835
Epoch: 80: loss=5.055, accuracy=0.836/0.833
Epoch: 100: loss=5.050, accuracy=0.836/0.838

Final Test: final accuray = 0.838
